In [ ]:
# Imports
%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib import pyplot as plt
import csv
import importlib
from tqdm import tqdm
from scripts import proj1_helpers, helpers
from scripts import implementation, feature_processing, k_fold, model_linear, model_logistic
import numpy as np

In [ ]:
train_path = '../data/train.csv'
test_path  = '../data/test.csv'
output_path = '../data/linear_processed_submission.csv'

In [ ]:
# loading data
y, X, idx = proj1_helpers.load_csv_data(train_path)
y_t, X_t, ids_t = proj1_helpers.load_csv_data(test_path)

In [ ]:
X_p = feature_processing.process_X(X)
X_t_p = feature_processing.process_X(X_t)

In [ ]:
#Linear
model = implementation.ridge_regression
loss = model_linear.compute_accuracy_loss

idx_min_l, rmse_all_l, lambdas_l = k_fold.cross_validation_select(X_p, y, model, loss, seed = 1,
                                                            k_fold = 3, N = 20, do_plot = True, do_tqdm = True)

In [ ]:
lambda_ = lambdas[idx_min]
print(lambda_)
w, l = implementation.ridge_regression(y, X_p, lambda_ = 0)

In [ ]:
#Logistic
y_01 = helpers.y_to_01(np.array([y]).T)
np.random.seed(1)
w0 = np.zeros((X_p.shape[1], 1))#np.random.randn(X_p.shape[1], 1)
model = implementation.reg_logistic_regression_newton_batch
model_args = {'initial_w': w0, 'max_iters': 100, 'gamma': 0.1, 'debug': False, 'batch_size': 50000}
#loss = model_logistic.reg_loss
loss = model_linear.compute_accuracy_loss

lambdas = np.logspace(-6, 1, 10)

idx_min, rmse_all, lambdas = k_fold.cross_validation_select(X_p, y_01, model, loss, kw_model = model_args, seed = 1,
                                                            k_fold = 3, lambdas = lambdas, do_plot = True, do_tqdm = False)

  0%|          | 0/100 [00:00<?, ?epoch/s]

In [ ]:
lambda_ = lambdas[idx_min]
print(lambda_)

In [ ]:
#w1, l = implementation.reg_logistic_regression(y_01, X_p, lambda_ = 0.1,
#                                                     initial_w = w0, max_iters = 100, gamma = 1e-6, debug = False)

In [198]:
w1, l = implementation.reg_logistic_regression_batch(y_01, X_p, lambda_ = 0.1,
                                                     initial_w = w0, batch_size = 50000,
                                                     max_iters = 300, gamma = 1e-5, debug = False)

 83%|████████▎ | 250/300 [00:39<00:07,  6.67epoch/s, acc=-0.78, grad=869, loss=1.15e+05, w=2.54]     


In [220]:
w3, l = implementation.reg_logistic_regression_newton_batch(y_01, X_p, lambda_ = 0.1,
                                                     initial_w = w0, batch_size = 50000,
                                                     max_iters = 50, gamma = 1e-1, debug = False)

 62%|██████▏   | 31/50 [00:12<00:07,  2.50epoch/s, acc=-0.8, grad=2.52, loss=1.11e+05, w=6.43] 


KeyboardInterrupt: 

In [219]:
y_pred = proj1_helpers.predict_labels(w3, X_t_p)
proj1_helpers.create_csv_submission(ids_t, y_pred, output_path)

In [150]:
tqdm.get_lock().locks = []